In [5]:
# Define constants
width, height = 200, 200  # Adjust these dimensions based on your requirements
channels = 1  # 1 for grayscale images, 3 for RGB images
num_classes = 7  # Adjust based on the number of age classes in your dataset
num_epochs = 6
# batch_size = 502
batch_size = 32

In [6]:
import tensorflow as tf

# Load the SavedModel
loaded_model = tf.keras.models.load_model('./saved_model/')

# Define a sample input (replace with your actual input shape)
sample_input = tf.ones((1, width, height, channels))

# Create a concrete function using tf.function
@tf.function(input_signature=[tf.TensorSpec(shape=(None, width, height, channels), dtype=tf.float32)])
def inference_fn(inputs):
    return loaded_model(inputs)

# Get the concrete function
concrete_func = inference_fn.get_concrete_function(sample_input)

# Convert the model to a frozen graph
frozen_graph_def = tf.compat.v1.graph_util.convert_variables_to_constants(
    tf.compat.v1.Session(graph=tf.Graph()), 
    concrete_func.graph.as_graph_def(), 
    [concrete_func.outputs[0].name.split(":")[0]]
)

# Save the frozen graph to a file
with tf.io.gfile.GFile('frozen_model.pb', 'wb') as f:
    f.write(frozen_graph_def.SerializeToString())


2023-12-15 12:45:33.566500: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-15 12:45:33.566586: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-15 12:45:33.566617: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-15 12:45:33.566837: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-15 12:45:33.566851: I tensorflow/core/common_runtime/gpu/gpu

In [8]:
import os 
import tensorflow as tf
# define the directory for .pb model
pb_model_path = "models"
# define the name of .pb model
pb_model_name = "mobilenet.pb"
# create directory for further converted model
os.makedirs(pb_model_path, exist_ok=True)
# get model TF graph
tf_model_graph = tf.function(lambda x: tf.tf_model(x))
# get concrete function
tf_model_graph = tf_model_graph.get_concrete_function(
    tf.TensorSpec( tf.tf_model.inputs[0].shape,  tf.tf_model.inputs[0].dtype))
# obtain frozen concrete function
frozen_tf_func = tf.compat.v1.graph_util.convert_variables_to_constants_v2(tf_model_graph)
# get frozen graph
frozen_tf_func.graph.as_graph_def()
# save full tf model
tf.io.write_graph(graph_or_graph_def=frozen_tf_func.graph,
                  logdir=pb_model_path,
                  name=pb_model_name,
                  as_text=False)

AttributeError: module 'tensorflow' has no attribute 'tf_model'

In [4]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

# Load your trained TensorFlow model
loaded_model = tf.saved_model.load('./saved_model/')

# Create a concrete function with input signature
concrete_func = tf.function(lambda x: model(x))
concrete_func = concrete_func.get_concrete_function(tf.TensorSpec(shape=(None,) + model.input_shape[1:], dtype=tf.float32))

# Convert the model to a TensorFlow graph and freeze it
frozen_func = convert_variables_to_constants_v2(concrete_func)
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir='./frozen_model',
                  name='frozen_model.pb',
                  as_text=False)


NameError: name 'model' is not defined